In [ ]:
%matplotlib inline
import holoviews as hv
hv.notebook_extension()
import sympy.interactive
sympy.interactive.init_printing('mathjax')
import sys
sys.path.append('/home/bnijholt/orbitalfield/')

In [ ]:
from ipyparallel import Client
client = Client()
dview = client[:]
lview = client.load_balanced_view()
len(dview)

In [ ]:
# %%px --local
import kwant
from types import SimpleNamespace
import sympy
from sympy.physics.quantum import TensorProduct as kr
import numpy as np
import holoviews as hv
from scipy.constants import hbar, m_e, eV, physical_constants
from discretizer import Discretizer, momentum_operators
k_B = physical_constants['Boltzmann constant in eV/K'][0] * 1000
sx, sy, sz = [sympy.physics.matrices.msigma(i) for i in range(1, 4)]
s0 = sympy.eye(2)
s0 = sympy.eye(2)
s0sz = np.kron(s0, sz)
s0s0 = np.kron(s0, s0)

# Parameters taken from arXiv:1204.2792
# All constant parameters, mostly fundamental constants, in a SimpleNamespace.
constants = SimpleNamespace(
    m=0.015 * m_e,  # effective mass in kg
    a=10,  # lattice spacing in nm
    g=50,  # Lande factor
    hbar=hbar,
    m_e=m_e,
    e=eV,
    eV=eV,
    meV=eV * 1e-3)

constants.t = (hbar ** 2 / (2 * constants.m)) * (1e18 / constants.meV)  # meV * nm^2
constants.mu_B = physical_constants['Bohr magneton'][0] / constants.meV


def make_params(alpha=20,
                B_x=0,
                B_y=0,
                B_z=0,
                Delta=0.25,
                mu=0,
                orbital=True,
                A_correction=True,
                t=constants.t,
                g=constants.g,
                mu_B=constants.mu_B,
                V=lambda x,y,z: 0,
                **kwargs):
    """Function that creates a namespace with parameters.

    Parameters:
    -----------
    alpha : float
        Spin-orbit coupling strength in units of meV*nm.
    B_x, B_y, B_z : float
        The magnetic field strength in the x, y and z direction in units of Tesla.
    Delta : float
        The superconducting gap in units of meV.
    mu : float
        The chemical potential in units of meV.
    orbital : bool
        Switches the orbital effects on and off.
    A_correction : bool
        Corrects for the net supercurrent flowing in the wire. If True, the
        current will be set to zero.
    t : float
        Hopping parameter in meV * nm^2.
    g : float
        Lande g factor.
    mu_B : float
        Bohr magneton in meV/K.
    V : function
        Function of spatial coordinates (x, y, z) with is added to mu.

    Returns:
    --------
    p : SimpleNamespace object
        A simple container that is used to store Hamiltonian parameters.
    """
    p = SimpleNamespace(t=t,
                        g=g,
                        mu_B=mu_B,
                        alpha=alpha,
                        B_x=B_x,
                        B_y=B_y,
                        B_z=B_z,
                        Delta=Delta,
                        mu=mu,
                        orbital=orbital,
                        A_correction=A_correction,
                        V=V,
                        **kwargs)
    return p


def cylinder_sector(r1, r2=0, L=1, L0=0, phi=360, angle=0, a=10):
    phi *= np.pi / 360
    angle *= np.pi / 180
    r1sq, r2sq = r1 ** 2, r2 ** 2
    def sector(pos):
        x, y, z = pos
        n = (y + 1j * z) * np.exp(1j * angle)
        y, z = n.real, n.imag
        rsq = y ** 2 + z ** 2
        return r2sq <= rsq < r1sq and z >= np.cos(phi) * np.sqrt(rsq) and L0 <= x < L
    r_mid = (r1 + r2) / 2
    return sector, (L - a, r_mid * np.sin(angle), r_mid * np.cos(angle))


def at_interface(site1, site2, shape1, shape2):
    return (shape1[0](site1.pos) and shape2[0](site2.pos)) or (shape2[0](site1.pos) and shape1[0](site2.pos))


def discretized_hamiltonian(a=10):
    k_x, k_y, k_z = momentum_operators
    t, B_x, B_y, B_z, mu_B, Delta, mu, alpha, g, V = sympy.symbols('t B_x B_y B_z mu_B Delta mu alpha g V', real=True)
    t_interface = sympy.symbols('t_interface', real=True)
    k =  sympy.sqrt(k_x**2+k_y**2+k_z**2)
    
    ham = ((t * k**2 - mu - V) * kr(s0, sz) +
               alpha * (k_y * kr(sx, sz) - k_x * kr(sy, sz)) +
               0.5 * g * mu_B * (B_x * kr(sx, s0) + B_y * kr(sy, s0) + B_z * kr(sz, s0)) +
               Delta * kr(s0, sx))
        
    args = dict(space_dependent={'V'}, lattice_constant=a, discrete_coordinates={'x', 'y', 'z'})

    tb_normal = Discretizer(ham.subs(Delta, 0), **args)
    tb_sc = Discretizer(ham, **args)
    tb_interface = Discretizer(ham.subs(t, t_interface), **args)
    return tb_normal, tb_sc, tb_interface


def make_3d_lead(a=10, r1=50, r2=70, phi=135, angle=0):
    tb_normal, tb_sc, tb_interface = discretized_hamiltonian(a)

    syst = kwant.Builder(kwant.TranslationalSymmetry((-a, 0, 0)))
    L = a
    shape_normal = cylinder_sector(r1=r1, angle=angle, L=L, a=a)
    shape_sc = cylinder_sector(r1=r2, r2=r1, phi=phi, angle=angle, L=L, a=a)

    lat = tb_normal.lattice
    sc_sites = list(syst.expand(lat.shape(*shape_sc)))
    
    syst[lat.shape(*shape_normal)] = tb_normal.onsite
    syst[lat.shape(*shape_sc)] = tb_sc.onsite

    def peierls(val, ind):
        def phase(s1, s2, p):
            x, y, z = s1.pos
            A = lambda p, x, y, z: [p.B_y * z - p.B_z * y, 0, p.B_x * y]
            A_site = A(p, x, y, z)[ind]
            if p.A_correction:
                A_sc = [A(p, *site.pos) for site in sc_sites]
                A_site -= np.mean(A_sc, axis=0)[ind]
            A_site *= a * 1e-18 * eV / hbar
            return np.cos(A_site) * s0s0 - 1j * np.sin(A_site) * s0sz

        def with_phase(s1, s2, p):
            if p.orbital:
                return phase(s1, s2, p).dot(val(s1, s2, p))
            else:
                return val(s1, s2, p)
        return with_phase

    for hop, val in tb_sc.hoppings.items():
        ind = np.argmax(hop.delta)
        syst[hop] = peierls(val, ind)
    
    # Hoppings at the barrier between wire and superconductor
    for hop, val in tb_interface.hoppings.items():
        hopping_iterator = ((i, j) for (i, j) in kwant.builder.HoppingKind(hop.delta, lat)(syst) 
                            if at_interface(i, j, shape_sc, shape_normal))
        ind = np.argmax(hop.delta)
        syst[hopping_iterator] = peierls(val, ind)

    return syst


def make_3d_wire(a=10, L=50, r1=50, r2=70, phi=135, angle=0):
    tb_normal, tb_sc, tb_interface = discretized_hamiltonian()
    syst = kwant.Builder()
    shape_normal = cylinder_sector(r1=r1, angle=angle, L=L, a=a)
    start_slice = cylinder_sector(r1=r2, r2=r1, phi=phi, angle=angle, L=a, a=a)
    end_slice = cylinder_sector(r1=r2, r2=r1, phi=phi, angle=angle, L0=L-a, L=L, a=a)

    lat = tb_normal.lattice
    syst[lat.shape(*shape_normal)] = tb_normal.onsite
#     syst[lat.shape(*start_slice)] = tb_sc.onsite
#     syst[lat.shape(*end_slice)] = tb_sc.onsite
    
    def peierls(val, ind):
        def phase(s1, s2, p):
            x, y, z = s1.pos
            A = lambda p, x, y, z: [p.B_y * z - p.B_z * y, 0, p.B_x * y]
            A_site = A(p, x, y, z)[ind]
            if p.A_correction:
                A_sc = [A(p, *site.pos) for site in sc_sites]
                A_site -= np.mean(A_sc, axis=0)[ind]
            A_site *= a * 1e-18 * eV / hbar
            return np.cos(A_site) * s0s0 - 1j * np.sin(A_site) * s0sz

        def with_phase(s1, s2, p):
            if p.orbital:
                return phase(s1, s2, p).dot(val(s1, s2, p))
            else:
                return val(s1, s2, p)
        return with_phase

    for hop, val in tb_normal.hoppings.items():
        ind = np.argmax(hop.delta)
        syst[hop] = peierls(val, ind)

#     # Hoppings at the barrier between wire and superconductor
#     for hop, val in tb_interface.hoppings.items():
#         hoppings_start = ((i, j) for (i, j) in kwant.builder.HoppingKind(hop.delta, lat)(syst)
#                              if at_interface(i, j, start_slice, shape_normal))
#         hoppings_end = ((i, j) for (i, j) in kwant.builder.HoppingKind(hop.delta, lat)(syst)
#                              if at_interface(i, j, end_slice, shape_normal))
#         ind = np.argmax(hop.delta)
#         syst[hoppings_start] = peierls(val, ind)
#         syst[hoppings_end] = peierls(val, ind)

    l_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 1]
    r_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 2]

    assert len(r_cut) == len(l_cut)
    num_orbs = 4
    dim = num_orbs * (len(l_cut) + len(r_cut))
    vlead = kwant.builder.SelfEnergyLead(lambda energy, args: np.zeros((dim, dim)), l_cut + r_cut)
    syst.leads.append(vlead)
    return syst, l_cut, r_cut

def make_scatter_syst(a=10, L=50, r1=50, r2=70, phi=135, angle=0):
    syst, l_cut, r_cut = make_3d_wire(a=a, L=L, r1=r1, r2=r2, phi=phi, angle=angle)
    lead = make_3d_lead(a=a, r1=r1, r2=r2, phi=phi, angle=angle)
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    syst = syst.finalized()
    r_cut_sites = [syst.sites.index(site) for site in r_cut]
    l_cut_sites = [syst.sites.index(site) for site in l_cut]
    
    def hopping(syst, args=()):
        ham_ = syst.hamiltonian_submatrix(args=args,
                                         to_sites=r_cut_sites,
                                         from_sites=l_cut_sites)
        return electron_subblock(ham_)
    
    return syst, hopping


def electron_subblock(mat):
    return mat[::2, ::2]


def matsubara_frequency(T, n):
    return (2*n + 1) * np.pi * T * 1j


def null_H(syst, T, n):
    en = matsubara_frequency(T, n)
    gf = kwant.greens_function(syst, en, [p], [0], [0], check_hermiticity=False)
    return np.linalg.inv(electron_subblock(gf.data))


def gf_from_H_0(H_0, t):
    H = np.copy(H_0)
    dim = t.shape[0]
    H[:dim, dim:] -= t.T.conj()
    H[dim:, :dim] -= t
    return np.linalg.inv(H)


def current_from_H_0(T, H_0s, H12, phase):
    t = H12 * np.exp(1j * phase)
    I = 0
    for H_0 in H_0s:
        gf = gf_from_H_0(H_0, t - H12)
        dim = t.shape[0]
        H12G21 = t.T.conj() @ gf[dim:, :dim]
        H21G12 = t @ gf[:dim, dim:]
        I += -4 * T * (np.trace(H21G12) - np.trace(H12G21)).imag
    return I


In [ ]:
# %%px --local
T = 0.060
matsfreqs = 2
syst, hopping = make_scatter_syst(r1=20, L=30)
p = make_params(Delta=0.25, alpha=0, A_correction=False, orbital=False, mu=110, t_interface=constants.t)

In [ ]:
kwant.plot(syst)

In [ ]:
%%opts Path [aspect='square']

bands = kwant.physics.Bands(syst.leads[1], args=[p])
ks = np.linspace(-2, 2, 100)
en = np.array([bands(k=k) for k in ks])
hv.Path((ks, en))[:, -20:20]

In [ ]:
H_0s = list(map(lambda n: null_H(syst, T, n), range(matsfreqs)))
H12 = hopping(syst, args=[p])
phases = np.linspace(-np.pi, np.pi, 51)
I = list(map(lambda phase: current_from_H_0(T, H_0s, H12, phase), phases))

In [ ]:
H_0s = lview.map_sync(lambda n: null_H(syst, T, n), range(matsfreqs))
H12 = hopping(syst, args=[p])
phases = np.linspace(-np.pi, np.pi, 51)
I = lview.map_sync(lambda phase: current_from_H_0(T, H_0s, H12, phase), phases)

In [ ]:
I = np.array(I)
hv.Curve((phases, I), kdims=['phase'], vdims=['$I$'])

# Debugging

In [ ]:
from types import SimpleNamespace
import kwant
import numpy as np
import holoviews as hv
hv.notebook_extension()

s0 = np.array([[1., 0.], [0., 1.]])
sx = np.array([[0., 1.], [1., 0.]])
sy = np.array([[0., -1j], [1j, 0.]])
sz = np.array([[1., 0.], [0., -1.]])

onsite = lambda site, p: -p.mu * sz
onsite_lead = lambda site, p: -p.mu * sz + p.delta * sx
hops = lambda site1, site2, p: -p.t * sz

def make_test_system(X, Y, Z):
    lat = kwant.lattice.general(np.eye(3))
    syst = kwant.Builder()
    range_X = range(X)
    range_Y = range(Y)
    range_Z = range(Z)
    syst[(lat(x, y, z) for x in range_X for y in range_Y for z in range_Z)] = onsite
    syst[lat.neighbors()] = hops

    l_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 0]
    r_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 1]

    num_orbs = 2
    dim = num_orbs * (len(l_cut) + len(r_cut))
    vlead = kwant.builder.SelfEnergyLead(lambda energy, args: np.zeros((dim, dim)), r_cut + l_cut)
    syst.leads.append(vlead)
    
    lead = kwant.Builder(kwant.TranslationalSymmetry((-1, 0, 0)))
    lead[(lat(0, y, z) for y in range_Y for z in range_Z)] = onsite_lead
    lead[lat.neighbors()] = hops
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    syst = syst.finalized()
    
    r_cut_sites = [syst.sites.index(site) for site in r_cut]
    l_cut_sites = [syst.sites.index(site) for site in l_cut]
    
    def hopping(syst, args=()):
        return syst.hamiltonian_submatrix(args=args,
                                          to_sites=l_cut_sites,
                                          from_sites=r_cut_sites)[::2, ::2]
    return syst, hopping


In [ ]:
def matsubara_frequency(T, n):
    return (2*n + 1) * np.pi * T * 1j


def null_H(syst, T, n):
    en = matsubara_frequency(T, n)
    gf = kwant.greens_function(syst, en, [p], [0], [0], check_hermiticity=False)
    return np.linalg.inv(gf.data[::2, ::2])


def gf_from_H_0(H_0, t):
    H = np.copy(H_0)
    dim = t.shape[0]
    H[:dim, dim:] -= t.T.conj()
    H[dim:, :dim] -= t
    return np.linalg.inv(H)


def current_from_H_0(T, H_0s, H12, phase):
    t = H12 * np.exp(1j * phase)
    I = 0
    for H_0 in H_0s:
        gf = gf_from_H_0(H_0, t - H12)
        dim = t.shape[0]
        H12G21 = t.T.conj() @ gf[dim:, :dim]
        H21G12 = t @ gf[:dim, dim:]
        I += -4 * T * (np.trace(H21G12) - np.trace(H12G21)).imag
    return I

delta = 0.01
T = 0.5 * delta
matsfreqs = 200


syst, hopping = make_test_system(X=4, Y=5, Z=5)
p = SimpleNamespace(mu=0., t=1., delta=1)

H12 = hopping(syst, [p])
phases = np.linspace(-np.pi, np.pi, 51)
H_0s = [null_H(syst, T, n) for n in range(matsfreqs)]
I = np.array([current_from_H_0(T, H_0s, H12, phase) for phase in phases])
hv.Curve(I, kdims=['phase'], vdims=['$I$'])

In [ ]:
kwant.plot(syst)

# Simpler way (but slightly slower)

In [ ]:
def null_G(syst, T, n):
    en = matsubara_frequency(T, n)
    gf = kwant.greens_function(syst, en, [p], [0], [0], check_hermiticity=False)
    return electron_subblock(gf.data)

def current_from_G_0(T, G_0s, H12, phase):
    t = H12 * np.exp(1j * phase)
    dim = t.shape[0]
    I = 0
    for G_0 in G_0s:
        V = np.zeros_like(G_0, dtype=complex)
        v = t - H12
        V[:dim, dim:] = v.T.conj()
        V[dim:, :dim] = v
        gf = np.linalg.solve(np.identity(2*dim) - G_0 @ V, G_0)
        H12G21 = t.T.conj() @ gf[dim:, :dim]
        H21G12 = t @ gf[:dim, dim:]
        I += -4 * T * (np.trace(H21G12) - np.trace(H12G21)).imag
    return I

G_0s = [null_G(syst, T, n) for n in range(matsfreqs)]
H12 = hopping(syst, args=[p])
phases = np.linspace(-np.pi, np.pi, 51)
I = [current_from_G_0(T, G_0s, H12, phase) for phase in phases]
I = np.array(I)
hv.Curve((phases, I), kdims=['phase'], vdims=['$I$'])

# Converging the above system to my system

In [ ]:
from types import SimpleNamespace
import kwant
import numpy as np
import holoviews as hv
hv.notebook_extension()

def make_test_system(X, Y, Z, a=10):
    tb_normal, tb_sc = discretized_hamiltonian(a)[:-1]
    lat = tb_normal.lattice
    syst = kwant.Builder()
    syst[(lat(x, y, z) for x in range(X) for y in range(Y) for z in range(Z))] = tb_normal.onsite

    for hop, val in tb_normal.hoppings.items():
        syst[hop] = val

    l_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 0]
    r_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 1]

    num_orbs = 4
    dim = num_orbs * (len(l_cut) + len(r_cut))
    vlead = kwant.builder.SelfEnergyLead(lambda energy, args: np.zeros((dim, dim)), r_cut + l_cut)
    syst.leads.append(vlead)
    
    lead = kwant.Builder(kwant.TranslationalSymmetry((-a, 0, 0)))
    lead[(lat(0, y, z) for y in range(Y) for z in range(Z))] = tb_sc.onsite

    for hop, val in tb_sc.hoppings.items():
        lead[hop] = val

    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    
    syst = syst.finalized()
    
    r_cut_sites = [syst.sites.index(site) for site in r_cut]
    l_cut_sites = [syst.sites.index(site) for site in l_cut]
    
    def hopping(syst, args=()):
        return syst.hamiltonian_submatrix(args=args,
                                          to_sites=l_cut_sites,
                                          from_sites=r_cut_sites)[::2, ::2]
    return syst, hopping


def matsubara_frequency(T, n):
    return (2*n + 1) * np.pi * T * 1j


def null_H(syst, T, n):
    en = matsubara_frequency(T, n)
    gf = kwant.greens_function(syst, en, [p], [0], [0], check_hermiticity=False)
    return np.linalg.inv(gf.data[::2, ::2])


def gf_from_H_0(H_0, t):
    H = np.copy(H_0)
    dim = t.shape[0]
    H[:dim, dim:] -= t.T.conj()
    H[dim:, :dim] -= t
    return np.linalg.inv(H)


def current_from_H_0(T, H_0s, H12, phase):
    t = H12 * np.exp(1j * phase)
    I = 0
    for H_0 in H_0s:
        gf = gf_from_H_0(H_0, t - H12)
        dim = t.shape[0]
        H12G21 = t.T.conj() @ gf[dim:, :dim]
        H21G12 = t @ gf[:dim, dim:]
        I += -4 * T * (np.trace(H21G12) - np.trace(H12G21)).imag
    return I


T = 60e-3
matsfreqs = 20

syst, hopping = make_test_system(X=4, Y=5, Z=5)
p = make_params(Delta=0.0, alpha=20, mu=15)

H12 = hopping(syst, [p])
phases = np.linspace(-np.pi, np.pi, 51)
H_0s = [null_H(syst, T, n) for n in range(matsfreqs)]
I = np.array([current_from_H_0(T, H_0s, H12, phase) for phase in phases])
hv.Curve(I, kdims=['phase'], vdims=['$I$'])

In [ ]:
%%opts Path [aspect='square']

bands = kwant.physics.Bands(syst.leads[1], args=[p])
ks = np.linspace(-2, 2, 100)
en = np.array([bands(k=k) for k in ks])
hv.Path((ks, en))[:, -20:20]

# Now not square cross-sectioned

In [ ]:
# %%px --local
from types import SimpleNamespace
import kwant
import numpy as np
import holoviews as hv

def make_test_system(a=10, L=50, r1=50, r2=70, phi=135, angle=0):
    tb_normal, tb_sc, tb_interface = discretized_hamiltonian(a)
    lat = tb_normal.lattice
    syst = kwant.Builder()
    
    shape_normal = cylinder_sector(r1=r1, angle=angle, L=L, a=a)
    syst[lat.shape(*shape_normal)] = tb_normal.onsite
    
    shape_sc = cylinder_sector(r1=r2, r2=r1, phi=phi, angle=angle, L=a, a=a)
    shape_sc2 = cylinder_sector(r1=r2, r2=r1, phi=phi, angle=angle, L0=L-a, L=L, a=a)
    
    for shape in [shape_sc, shape_sc2]:
        syst[lat.shape(*shape)] = tb_sc.onsite

    def peierls(val, ind):
        def phase(s1, s2, p):
            x, y, z = s1.pos
            A_site = [p.B_y * z - p.B_z * y, 0, p.B_x * y][ind]
            A_site *= a * 1e-18 * eV / hbar
            return np.cos(A_site) * s0s0 - 1j * np.sin(A_site) * s0sz

        def with_phase(s1, s2, p):
            if p.orbital:
                return phase(s1, s2, p).dot(val(s1, s2, p))
            else:
                return val(s1, s2, p)
        return with_phase

    for hop, val in tb_normal.hoppings.items():
        ind = np.argmax(hop.delta)
        syst[hop] = peierls(val, ind)

    l_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 1]
    r_cut = [lat(x, y, z) for x, y, z in [i.tag for i in syst.sites()] if x == 2]

    num_orbs = 4
    dim = num_orbs * (len(l_cut) + len(r_cut))
    vlead = kwant.builder.SelfEnergyLead(lambda energy, args: np.zeros((dim, dim)), r_cut + l_cut)
    syst.leads.append(vlead)
    
    lead = kwant.Builder(kwant.TranslationalSymmetry((-a, 0, 0)))
    
    shape_normal_lead = cylinder_sector(r1=r1, angle=angle, L=a, a=a)
    lead[lat.shape(*shape_normal_lead)] = tb_normal.onsite
    lead[lat.shape(*shape_sc)] = tb_sc.onsite
    
    for hop, val in tb_sc.hoppings.items():
        ind = np.argmax(hop.delta)
        lead[hop] = peierls(val, ind)

    for hop, val in tb_interface.hoppings.items():
        ind = np.argmax(hop.delta)
        for part, shape1, shape2 in [(syst, shape_sc, shape_normal), 
                                     (syst, shape_sc2, shape_normal), 
                                     (lead, shape_sc, shape_normal_lead)]:
            hoppingkind = kwant.builder.HoppingKind(hop.delta, lat)(part)
            hopping_iterator = ((i, j) for (i, j) in hoppingkind if at_interface(i, j, shape1, shape2))
            part[hopping_iterator] = peierls(val, ind)   
        
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    
    syst = syst.finalized()
    
    r_cut_sites = [syst.sites.index(site) for site in r_cut]
    l_cut_sites = [syst.sites.index(site) for site in l_cut]
    
    def hopping(syst, args=()):
        return syst.hamiltonian_submatrix(args=args,
                                          to_sites=l_cut_sites,
                                          from_sites=r_cut_sites)[::2, ::2]
    return syst, hopping

T = 60e-3
matsfreqs = 3

syst, hopping = make_test_system(r1=50, r2=70)
p = make_params(Delta=0, alpha=20, mu=15, t_interface=constants.t)

H12 = hopping(syst, [p])
phases = np.linspace(-np.pi, np.pi, 51)
H_0s = [null_H(syst, T, n) for n in range(matsfreqs)]
I = np.array([current_from_H_0(T, H_0s, H12, phase) for phase in phases])
hv.Curve(I, kdims=['phase'], vdims=['$I$'])


In [ ]:
kwant.plot(syst)